In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset
from logger_utils import custom_sink
from loguru import logger
%matplotlib inline

logger.remove()
logger.add(custom_sink)
functionLogger = logger.bind(format="function_inputs")

train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[1]*train_set_x_orig.shape[2]*train_set_x_orig.shape[3],train_set_x_orig.shape[0])
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[1]*test_set_x_orig.shape[2]*test_set_x_orig.shape[3],test_set_x_orig.shape[0])

train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.

def sigmoid(z): return 1/(1+np.exp(-z))

def initialize_with_zeros(dim): return (np.zeros((dim, 1)), 0)



In [ ]:

def propagate(w, b, X, Y):
    # functionLogger.bind(w=w,b=b,X=X,Y=Y).debug("Entering propagate with the following args:")
    m = X.shape[1]
    A = sigmoid(np.dot(w.T,X) + b)              # compute activation
    # functionLogger.bind(wT=w.T,wTX=np.dot(w.T,X),b=b, A=A).debug("Performing Dot product with the following:")
    cost = np.sum((- np.log(A))*Y + (-np.log(1-A))*(1-Y))/m  # compute cost
    dw = (np.dot(X,(A-Y).T))/m
    db = (np.sum(A-Y))/m
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    grads = {"dw": dw,
             "db": db}
    return grads, cost

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    # functionLogger.bind(w=w,b=b,X=X,Y=Y).debug("Entering optimize with the following args:")
    costs = []
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]
        w = w - (learning_rate*dw)
        b = b - (learning_rate*db)
        if i % 100 == 0: costs.append(cost)
        if print_cost and i % 100 == 0: print ("Cost after iteration %i: %f" %(i, cost))
    params = {"w": w,
              "b": b}
    grads = {"dw": dw,
             "db": db}
    return params, grads, costs

def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    A = sigmoid(np.dot(w.T, X) + b)
    Y_prediction = (A >= 0.5) * 1.0
    functionLogger.bind(wT=w.T,X=X,wTX=np.dot(w.T,X),A=A,Y_prediction=Y_prediction).debug("Prediction results: ")
    assert(Y_prediction.shape == (1, m))
    return Y_prediction

def model(X_train, Y_train, X_test, Y_test, num_iterations=500, learning_rate=0.01, print_cost=False):
    w, b = initialize_with_zeros(X_train.shape[0])
    params, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate)
    w, b = params.values()
    Y_prediction_train = predict(w, b, X_train)
    Y_prediction_test = predict(w, b, X_test)
    print("train accuracy: {} %", (100-np.mean(np.abs(Y_prediction_train-Y_train))* 100))
    print("test accuracy: {} %", (100-np.mean(np.abs(Y_prediction_test-Y_test))* 100))
    d = {
        "costs": costs,
        "Y_prediction_test": Y_prediction_test,
        "Y_prediction_train": Y_prediction_train,
        "w": w,
        "b": b,
        "learning_rate": learning_rate,
        "num_iterations": num_iterations
    }
    # logger.debug("d:\n {}", d)
    return d

In [3]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 500, learning_rate = 0.008, print_cost = False)

test_set_y

train accuracy: {} % 63.15789473684211
test accuracy: {} % 66.0


array([[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
        0, 0, 1, 1, 1, 0]])